# Custom Object Detection
## Colab Trainer

This notebook is for training your custom model after running [this](https://github.com/theneuralbeing/object_detection_template/blob/master/data_preprocessing/Preprocess_Data.ipynb) notebook locally on your computer which generates the tf record files for your custom dataset. You can run all these cells in order and follow the steps written as comments and markdown cells along the way.

Colab VM Project Structure

```
content/
        ├─ data/
        │    ├── label_map.pbtxt
        │    ├── test.record
        │    └── train.record
        └─ models/
             ├─ research/
             │      ├── fine_tuned_model/
             │      │         ├── frozen_inference_graph.pb
             │      │         └── ...
             │      │         
             │      ├── pretrained_model/
             │      │         ├── frozen_inference_graph.pb
             │      │         └── ...
             │      │         
             │      ├── object_detection/
             │      │         ├── utils/
             │      │         ├── samples/
             │      │         │      ├── samples/ 
             │      │         │      │       ├── configs/             
             │      │         │      │       │     ├── ssd_mobilenet_v2_coco.config
             │      │         │      │       │     ├── rfcn_resnet101_pets.config
             │      │         │      │       │     └── ...
             │      │         │      │       └── ... 
             │      │         │      └── ...                                
             │      │         ├── export_inference_graph.py
             │      │         ├── model_main.py
             │      │         └── ...
             │      │         
             │      ├── training/
             │      │         ├── events.out.tfevents.xxxxx
             │      │         └── ...               
             │      └── ...
             └── ...




## Installing Prerequisites and Importing the Libraries

In [0]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -qq Cython contextlib2 pillow lxml matplotlib

!pip install -qq pycocotools

In [0]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io
import tensorflow.compat.v1 as tf

from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

In [0]:
# Object Detection API works in Tensorflow v 1.15 (it is remove in v 2.0)
print(tf.__version__)

## Getting preprocess data

In [0]:
# Upload your data.zip file which contains the record files and label map
files.upload()

In [0]:
!mkdir data
!unzip data.zip -d data/

## Downloading and Installing Tensorflow Object Detection API

In [0]:
# Downlaods Tensorflow Object Detection API
%cd /content
!git clone --q https://github.com/tensorflow/models.git

In [0]:
%cd /content/models/research
#compiling the proto buffers (not important to understand for this project but you can learn more about them here: https://developers.google.com/protocol-buffers/)
!protoc object_detection/protos/*.proto --python_out=.

# exports the PYTHONPATH environment variable with the reasearch and slim folders' paths
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [0]:
# testing the model builder
!python3 object_detection/builders/model_builder_test.py

## Getting the pretrained model

Check other models from [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models).

In [0]:
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
    }
}

# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'


In [0]:
%cd /content/models/research

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#the distination folder where the model will be saved
fine_tune_dir = '/content/models/research/pretrained_model'

#checks if the model has already been downloaded
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the file and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(fine_tune_dir)):
    shutil.rmtree(fine_tune_dir)
os.rename(MODEL, fine_tune_dir)

In [0]:
# checking the content of the pretrained model.
# this is the directory of the "fine_tune_checkpoint" that is used in the config file.
!echo {fine_tune_dir}
!ls -alh {fine_tune_dir}

## Configuring Training Pipeline

Editing the configuration file to add the path for the TFRecords files, pbtxt,  batch_size, num_steps, num_classes.

Any image augmentation, hyperparemeter tunning (drop out, batch normalization... etc) can be editted here in the configuration file

In [0]:
#the path to the folder containing all the sample config files
CONFIG_BASE = "/content/models/research/object_detection/samples/configs/"

#path to the specified model's config file
model_pipline = os.path.join(CONFIG_BASE, pipeline_file)
model_pipline

In [0]:
%%writefile {model_pipline}

model {
  ssd {
    num_classes: 1 # number of classes to be detected
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    # all images will be resized to the below W x H.
    image_resizer { 
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: true # to counter over fitting. you can also try tweaking its probability below
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
            # weight: 0.00004
            weight: 0.001 # higher regularizition to counter overfitting
          }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            # weight: 0.00004
            weight: 0.001 # higher regularizition to counter overfitting
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000 
        iou_threshold: 0.95
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        
        # adjust this to the max number of objects per class.
        max_detections_per_class: 4
        # max number of detections among all classes
        max_total_detections: 4
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 16 # training batch size
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.003
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }

  # the path to the pretrained model. 
  fine_tune_checkpoint: "/content/models/research/pretrained_model/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  # edit the num_steps to your required amount of steps. Predefining steps
  # will stop the learning rate from decaying. You can also remove the line to 
  # train indefinitely
  # num_steps: 10000
  

  # data augmentaion is done here, you can remove or add more.
  # They will help the model generalize but the training time will increase greatly by using more data augmentation.
  # Check this link to add more image augmentation: https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto
  
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    #path to the training TFRecord
    input_path: "/content/data/train.record"
  }
  #path to the label map 
  label_map_path: "/content/data/label_map.pbtxt"
}

eval_config: {
  # the number of images in your "testing" data (was 600 but we removed one above :) )
  num_examples: 30
  # the number of images to disply in Tensorboard while training
  num_visualizations: 5

  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  # max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
      
    #path to the testing TFRecord
    input_path: "/content/data/test.record"
  }
  #path to the label map 
  label_map_path: "/content/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

In [0]:
# where the model will be saved at each checkpoint while training 
model_dir = 'training/'

# Optionally: remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Tensorboard

In [0]:
# downloading ngrok to be able to access tensorboard on google colab
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
# the logs that are created while training 
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
# The link to tensorboard.
# works after the training starts.

### note: if you didnt get a link as output, rerun this cell and the one above
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])" 

## Training

In [0]:
!python3 /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={model_pipline}\
    --model_dir={model_dir} \
    --alsologtostderr \

## Exporting the Trained Model as a Graph

In [0]:
# the location where the exported model will be saved in.
output_directory = '/content/models/research/fine_tuned_model'

# goes through the model is the training/ dir and gets the last one.
# you could choose a specfic one instead of the last
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

#exports the model specifed and inference graph
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={model_pipline} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [0]:
# download the frozen model that is needed for inference
files.download(output_directory + '/frozen_inference_graph.pb')

## Performing Inference
To perform inference locally on your computer, follow these steps

1. Ideally create a separate environment with Tensorflow version 1.15 installed and then `pip install object_detection`
2. Open the [`inference_webcam.py`]()
3. Edit the path to the frozen graph in the file
4. If you remember our label_map.pbtxt was of the format
```
item {
    id: 1
    name: 'LABEL_1'
}
item {
    id: 2
    name: 'LABEL_2'
}
```
Now edit the `category_index` dictionary in the inference file in the following format
```
category_index = {1: {'id': 1, 'name': 'LABEL_1'}, 2: {'id': 2, 'name': 'LABEL_2'}}
```
5. Finally run `inference_webcam.py` in the terminal and your custom object detection model is ready.